# 3.6 损失函数

在深度学习广为使用的今天，我们可以在脑海里清晰的知道，一个模型想要达到很好的效果需要**学习**，也就是我们常说的训练。一个好的训练离不开优质的负反馈，这里的损失函数就是模型的负反馈。

![](https://admin-hwj.oss-cn-beijing.aliyuncs.com/img/202410182128540.png)

所以在PyTorch中，损失函数是必不可少的。它是数据输入到模型当中，产生的结果与真实标签的评价指标，我们的模型可以按照损失函数的目标来做出改进。

下面我们将开始探索PyTorch的所拥有的损失函数。这里将列出PyTorch中常用的损失函数（一般通过torch.nn调用），并详细介绍每个损失函数的功能介绍、数学公式和调用代码。当然，PyTorch的损失函数还远不止这些，在解决实际问题的过程中需要进一步探索、借鉴现有工作，或者设计自己的损失函数。

经过本节的学习，你将收获：

- 在深度学习中常见的损失函数及其定义方式
- PyTorch中损失函数的调用


## 3.5.1 二分类交叉熵损失函数

In [ ]:
import torch
from torch import nn as nn
nn.BCELoss(weight=None,size_average = None,reduce=None,reduction='mean')

**功能**：计算二分类任务时的交叉熵（Cross Entropy）函数。在二分类中，label是{0,1}。对于进入交叉熵函数的input为概率分布的形式。一般来说，input为sigmoid激活层的输出，或者softmax的输出。

**主要参数**：

`weight`:每个类别的loss设置权值

`size_average`:数据为bool，为True时，返回的loss为平均值；为False时，返回的各样本的loss之和。

`reduce`:数据类型为bool，为True时，loss的返回是标量。

计算公式如下：
$
\ell(x, y)=\left\{\begin{array}{ll}
\operatorname{mean}(L), & \text { if reduction }=\text { 'mean' } \\
\operatorname{sum}(L), & \text { if reduction }=\text { 'sum' }
\end{array}\right.
$

In [ ]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3,requires_grad=True)
target = torch.empty(3).random_(2)
output = (loss(m(input),target))
output.backward()
print('BCELoss损失函数的计算结果为',output)

## 3.5.2 交叉熵损失函数

In [14]:
torch.nn.CrossEntropyLoss(weight = None,size_average = None,ignore_index=-100,reduce=None,reduction='mean')

CrossEntropyLoss()

**功能**：计算交叉熵函数

**主要参数**：  

`weight`:每个类别的loss设置权值。

`size_average`:数据为bool，为True时，返回的loss为平均值；为False时，返回的各样本的loss之和。

`ignore_index`:忽略某个类的损失函数。

`reduce`:数据类型为bool，为True时，loss的返回是标量。

计算公式如下：
$
\operatorname{loss}(x, \text { class })=-\log \left(\frac{\exp (x[\text { class }])}{\sum_{j} \exp (x[j])}\right)=-x[\text { class }]+\log \left(\sum_{j} \exp (x[j])\right)
$

In [20]:
loss = nn.CrossEntropyLoss()
m = nn.Sigmoid()
input = torch.randn(3,5,requires_grad=True)
target = torch.empty(3,dtype = torch.long).random_(5)
output = loss(input,target)
output.backward()
print(output)

tensor(2.0803, grad_fn=<NllLossBackward0>)


## 3.5.3 L1损失函数

In [24]:
torch.nn.L1Loss(size_average=None,reduce=None,reduction='mean')

L1Loss()

**功能：** 计算输出`y`和真实标签`target`之间的差值的绝对值。

我们需要知道的是，`reduction`参数决定了计算模式。有三种计算模式可选：none：逐个元素计算。
sum：所有元素求和，返回标量。
mean：加权平均，返回标量。 
如果选择`none`，那么返回的结果是和输入元素相同尺寸的。默认计算方式是求平均。

**计算公式如下：**
$
L_{n} = |x_{n}-y_{n}|
$

In [26]:
loss = nn.L1Loss()
input= torch.randn(3,5,requires_grad= True)
target = torch.randn(3,5)
output = loss(input,target)
output.backward()
output

tensor(1.4400, grad_fn=<MeanBackward0>)

## 3.5.4 MSE损失函数

In [ ]:
torch.nn.MSE(size_average=None,reduce = None,reduction='mean')

**功能：** 计算输出`y`和真实标签`target`之差的平方。

和`L1Loss`一样，`MSELoss`损失函数中，`reduction`参数决定了计算模式。有三种计算模式可选：none：逐个元素计算。
sum：所有元素求和，返回标量。默认计算方式是求平均。

**计算公式如下：**

$
l_{n}=\left(x_{n}-y_{n}\right)^{2}
$

In [37]:
loss = nn.MSELoss()
input = torch.arange(1,6,dtype=torch.float)
target = torch.tensor([3,3,3,3,3],dtype = torch.float)
output = loss(input,target)
output

tensor(2.)

## 3.5.5 平滑L1 (Smooth L1)损失函数

In [ ]:
torch.nn.Smooth(size_average=None,reduce=None,reduction=None,beta=1.0)

**功能：** L1的平滑输出，其功能是减轻离群点带来的影响

`reduction`参数决定了计算模式。有三种计算模式可选：none：逐个元素计算。
sum：所有元素求和，返回标量。默认计算方式是求平均。

**提醒：** 之后的损失函数中，关于`reduction` 这个参数依旧会存在。所以，之后就不再单独说明。

**计算公式如下：**
$
\operatorname{loss}(x, y)=\frac{1}{n} \sum_{i=1}^{n} z_{i}
$
其中，
$
z_{i}=\left\{\begin{array}{ll}
0.5\left(x_{i}-y_{i}\right)^{2}, & \text { if }\left|x_{i}-y_{i}\right|<1 \\
\left|x_{i}-y_{i}\right|-0.5, & \text { otherwise }
\end{array}\right.
$